In [1]:
import tensorflow as tf

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import glob
import io
import base64
import numpy as np
import gym as gym
from gym.wrappers import Monitor
from gym import logger as gymlogger
gymlogger.set_level(40) #error only
from collections import namedtuple, deque
import matplotlib
import matplotlib.pyplot as plt
import time

from IPython.display import HTML
from IPython import display as ipythondisplay
import random
import copy
import tensorflow.contrib.eager as tfe
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
tf.enable_eager_execution()

# %matplotlib inline


In [2]:
#  _____            _     
# |_   _|___   ___ | |___ 
#   | | / _ \ / _ \| / __|
#   | || (_) | (_) | \__ \
#   |_| \___/ \___/|_|___/
#                         
# Just run it. Epsilon is the same as before

class Epsilon(object):
    def __init__(self, start=1.0, end=0.01, update_decrement=0.01):
        self._start = start
        self._end = end
        self._update_decrement = update_decrement
        self._value = self._start
        self.isTraining = True
    
    def decrement(self, count=1):
        self._value = max(self._end, self._value - self._update_decrement*count)
        return self
        
    def value(self):
        if not self.isTraining:
            return 0.0
        else:
            return self._value


          
class RewardHistory(list):
    def plot(self):
        R = np.array(self)
        mu = np.mean(R, axis=0)
        std = np.std(R, axis=0)
        f, axarr = plt.subplots(1, 2)
        f.set_figheight(5)
        f.set_figwidth(20)
        f.subplots_adjust(hspace=0.2)
        axarr[0].plot(mu)
        axarr[0].set_title('Mean reward')
        axarr[0].set_xlabel("Episodes")
        axarr[0].set_ylabel("Reward")
        axarr[1].set_title('Std')
        axarr[1].set_xlabel("Episodes")
        axarr[1].set_ylabel("std")
        axarr[1].plot(std)
        plt.show()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
    

def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env


In [3]:

from collections import namedtuple
import random
import numpy as np

Transition = namedtuple('Transition',["s", "a", "s_1", "r", "done"])
Transition

__main__.Transition

In [4]:

from collections import namedtuple
import random
import numpy as np

Transition = namedtuple('Transition',["s", "a", "s_1", "r", "done"])

class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.index = 0
        
    def push(self, item):
        """save a transition"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.index] = item
        self.index = (self.index+1) % self.capacity
        
    def sample(self, batch_size):
        out = random.sample(self.memory, batch_size)
        batched = Transition(*zip(*out))
        s = np.array(list(batched.s))
        a = np.expand_dims(np.array(list(batched.a)), axis = 1)
        s_1 = np.array(list(batched.s_1))
        r = np.expand_dims(np.array(list(batched.r)),axis = 1)
        done = np.expand_dims(np.array(list(batched.done)),axis = 1)
        
        return [s, a, s_1, r, done]
    
    def __len__(self):
        return len(self.memory)
    
    def __str__(self):
        result = []
        for i in range(self.__len__()):
            result.append(self.memory[i].__str__() + " \n")
        return "".join(result)

In [5]:
class LinearModel(tf.keras.Model):
    def __init__(self, nb_actions = 2):
        super().__init__(self)
        self.dense1 = tf.keras.layers.Dense(units = 64,activation = tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units = 128,activation = tf.nn.relu)
        self.dense22 = tf.keras.layers.Dense(units = 128,activation = tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(units = nb_actions)
        
    def call(self, input):
        result = self.dense1(input)
        result = self.dense2(result)
#         result = self.dense22(result)
        result = self.dense3(result)
        return result
nb_actions = 2




In [6]:
class DQN_LinearLearner(object):
    def __init__(self, env = None, Double_Q = False):
        self.env = env
        self.epsilon = Epsilon(start = 0.9, end = 0.01, update_decrement = 0.0001)
        self.gamma = 0.999
        self.train_q_per_step = 1 # Train every 4 steps
        self.train_q_batch_size = 256 
        self.steps_before_training = 1000 # Store more memory to initialize
        self.target_q_update_frequency = 100 # Copy Q to Qt every 100 steps
        self.memory = ReplayMemory(capacity=10000)
        self.optimizer = tf.train.AdamOptimizer(learning_rate= 0.0001)
        self.Q = LinearModel(env.action_space.n)
        self.Qt = LinearModel(env.action_space.n)
        self.Qt.set_weights(self.Q.get_weights())
        
        self.reset()
        self.episode_rewards = []
        self.epsilon_log = []
    
    def state_to_tensor(self,s):
        x = tf.constant(s, dtype = 'float32')
        x = tf.expand_dims(x, axis = 0)
        return x
    
    def get_actions(self, s):
        if np.random.rand() > self.epsilon.value():
            #                                  
            #  _____            _       
            # |_   _|___     __| | ___  
            #   | | / _ \   / _` |/ _ \ 
            #   | || (_) | | (_| | (_) |
            #   |_| \___/   \__,_|\___/ 
            #                           
            # - convert state to tensor to input into the Q NN
            # - get Q NN to calculate all the action values for this state, Q(s, A)
            # - select the greedy action from the NN outputs
            #action = self.env.action_space.sample()
            
            s = self.state_to_tensor(s)
            actions = self.Q(s).numpy()
            action = np.argmax(actions)    
        else:
            action = self.env.action_space.sample()
        return action
    
    def reset(self):
        self.s = self.env.reset()
        
    def train(self,nb_episodes = 1, watch = False):
        self.epsilon.isTraining = True
        step = 0
        
        for eps in range(nb_episodes):
            self.reset()
            episode_reward = 0
            
            while True:

                action = self.get_actions(self.s)
                s_1, r, done, _ = self.env.step(action)
                
                       
                # ReDesign Reward
                r0 = 0
                if r > -1:
                    r0 = r +10 * 200
                    
                r = abs(s_1[0] - (-0.5)) + r0
                
                memory_done = done
#                 if done and episode_reward >= 199:
#                   memory_done = False

                #  - store transition into memory  
                transition = Transition(self.s, action, s_1, r, memory_done)
                self.memory.push(transition)
                
                episode_reward += r
                step += 1
                self.s = s_1
                
                if done:
                    break;
                    
                if step % self.train_q_per_step == 0 and step > self.steps_before_training:
                    self.train_q()
                    
                if step % self.target_q_update_frequency == 0 and step > self.steps_before_training:
                    self.update_target_q()
                    
            if step > self.steps_before_training:
                self.epsilon.decrement()
            self.episode_rewards.append(episode_reward)
            self.epsilon_log.append(self.epsilon.value())
        print('Total steps:', step)
        
    def train_q(self):
        if self.train_q_batch_size >= len(self.memory):
            return
        
        s, a, s_1, r, done = self.memory.sample(self.train_q_batch_size)

        

        
        s = tf.constant(s, dtype = tf.float32)
        a = tf.squeeze(tf.constant(a, dtype = tf.int32))
        a_one_hot = tf.one_hot(tf.squeeze(a),depth = self.env.action_space.n, dtype = tf.float32)
        s_1 = tf.constant(s_1, dtype = tf.float32)
        r = tf.constant(r, dtype = tf.float32)
        done = tf.constant(1-done, dtype=tf.float32)

        
        # Calculate q values using Qt (Q target network)  -> Qt
        Qt = self.Qt(s_1)
        
        # Calculate the targets using the bellman equation: targets = r + done * gamma * Max_of_Qt    # why is done part of this equation?
        target = r + done * self.gamma * tf.reduce_max(Qt,axis = 1, keep_dims=True)
        
        # Start tracking gradients
        with tfe.GradientTape() as tape:
            # run the model with the inputs (states)
            q = self.Q(s)
            
            # y_hat = q*one_hot_actions
            y_hat = tf.reduce_sum(q * a_one_hot, keep_dims=True, reduction_indices = 1)
            
            # define your loss function between y_hat and targets
            loss = (y_hat - target) ** 2
            loss = tf.reduce_mean(loss)
            
        grads = tape.gradient(loss, self.Q.variables)
        self.optimizer.apply_gradients(zip(grads, self.Q.variables),
                                  global_step= tf.train.get_or_create_global_step())
    
    def update_target_q(self):
        self.Qt.set_weights(self.Q.get_weights())
        
    def run(self, env):
        self.env = env
        self.reset()
        self.epsilon.isTraining = False
        episode_reward = 0
        
        while True:
            self.env.render()
            action = self.get_actions(self.s)
            s_1, r, done, _ = self.env.step(action)
            
            r0 = 0
            if r > -1:
                r0 = r +10 * 200
                    
            r = abs(s_1[0] - (-0.5)) + r0
            
            episode_reward += r
            self.s = s_1
            
            
#             if steps >= epochs:
#                 break
            if done:
                break;
        
        self.env.close()
        print("Total Reward: ", episode_reward)
            
#     def run(self, env):
#         self.env = env
#         self.reset()
#         self.epsilon.isTraining = False
#         epsode_reward = 0
        
#         while True:
#             self.env.render()
#             action = self.get_actions(self.s)
#             s_1, r, done, _ = self.env.step(action)
#             epsode_reward += r
#             self.s = s_1
            
#             if done:
#                 break
                
#         self.env.close()
#         print('total reward:', epsode_reward)
        
        
        
        

In [7]:
import datetime

In [8]:
t1 = datetime.datetime.now()
reward_history = RewardHistory()
for i in range(1):
    env = gym.make("MountainCar-v0")
    learner = DQN_LinearLearner(env)
    learner.train(10000)
    reward_history.append(learner.episode_rewards)
    print(np.max(reward_history))
t2 = datetime.datetime.now()

print('total time: ',t2 - t1)

Instructions for updating:
Colocations handled automatically by placer.
Total steps: 1999661
79.47094315193993
total time:  7:02:48.918926


In [192]:
learner.Q.save_weights('/Users/deanlee/Desktop/jupyter/DQN weights/Q_weights.ckpt')

Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


In [11]:
learner.Q.load_weights('/Users/deanlee/Desktop/jupyter/DQN weights/Q_weights.ckpt')
learner.Qt.load_weights('/Users/deanlee/Desktop/jupyter/DQN weights/Q_weights.ckpt')

In [13]:
learner.train(100)

Instructions for updating:
Colocations handled automatically by placer.
Total steps: 20000


In [16]:
np.max(learner.episode_rewards)

33.76814212887136

In [9]:
env = wrap_env(gym.make("MountainCar-v0"))
for i in range(1):
    learner.run(env)
    show_video()
    

Total Reward:  38.02977596099416


In [159]:
env = wrap_env(gym.make("MountainCar-v0"))
for i in range(1):
    learner.run(env)
    show_video()
    

Total Reward:  42.751625774900106


In [163]:
s, a, s_1, r, done = learner.memory.sample(200)

In [168]:
s[np.where(done == True)[0][0]]

array([-0.72285768,  0.01005355])

In [177]:
env.step(1)

(array([-5.88208399e-01,  4.85113376e-04]), -1.0, False, {})

In [178]:
for i in range(200):
    s_1, r, done, _ = env.step(1)
    print(s_1)

[-0.58724174  0.00096666]
[-0.58580066  0.00144108]
[-0.58389576  0.0019049 ]
[-0.5815411   0.00235466]
[-0.57875406  0.00278704]
[-0.57555523  0.00319882]
[-0.57196831  0.00358692]
[-0.56801989  0.00394842]
[-0.5637393   0.00428059]
[-0.55915838  0.00458092]
[-0.55431127  0.00484711]
[-0.54923414  0.00507713]
[-0.54396493  0.00526921]
[-0.53854308  0.00542186]
[-0.53300918  0.0055339 ]
[-0.52740471  0.00560447]
[-0.52177169  0.00563301]
[-0.51615238  0.00561931]
[-0.51058891  0.00556347]
[-0.505123    0.00546592]
[-0.49979558  0.00532742]
[-0.49464653  0.00514905]
[-0.48971435  0.00493218]
[-0.48503586  0.00467848]
[-0.48064596  0.00438991]
[-0.47657731  0.00406865]
[-0.47286015  0.00371716]
[-0.46952206  0.00333809]
[-0.46658777  0.00293429]
[-0.46407899  0.00250879]
[-0.46201423  0.00206476]
[-0.46040873  0.0016055 ]
[-0.45927433  0.0011344 ]
[-0.45861937  0.00065496]
[-4.58448674e-01  1.70694723e-04]
[-4.58763500e-01 -3.14825991e-04]
[-0.45956153 -0.00079803]
[-0.46083689 -0.001275

In [ ]:
env.

In [108]:
s_1

array([[-0.56914189, -0.00557957]])

In [109]:
abs(s_1[0][0] - (-1.2))

0.6308581088057639

In [101]:
env.step(0)

(array([-0.51999196, -0.00562102]), -1.0, True, {'TimeLimit.truncated': True})